# Multiple Linear Regression for Canonical Proportion as a Predictor of Standardized Speech Assessments

The following notebook contains code to perform multiple linear regression for canonical proportion (CP) as a predictor of the following standardized speech assessments:
- GFTA
- CTOPP
- PPVT
- NWR
- RWR

The following variables were used as covariates:
- gender (dummy)
- age
- maternal education

## GFTA-2

### Unweighted, Unscaled (Baseline vs. Expanded)

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['GFTA_Standard', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    return model

# Fit the baseline model for GFTA
gfta_baseline_model = fit_model('GFTA_Standard', baseline_predictors)

# Fit the expanded model for GFTA (adding canonical_proportion)
gfta_expanded_model = fit_model('GFTA_Standard', expanded_predictors)

# Log-likelihoods of both models
log_likelihood_baseline = gfta_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = gfta_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(gfta_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(gfta_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of GFTA_Standard")
print(gfta_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of GFTA_Standard")
print(gfta_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


### Weighted, Scaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['GFTA_Standard', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['GFTA_Standard_scaled'] = scaler.fit_transform(df_clean[['GFTA_Standard']])
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Calculate a weight for each child based on their number of canonical and noncanonical clips
# We will add these columns together and normalize it so the sum of weights is 1
df_clean['clip_count'] = df_clean[['0', '1']].sum(axis=1)

# Normalize the weight column to ensure it sums to 1 (optional step)
df_clean['weights'] = df_clean['clip_count'] / df_clean['clip_count'].sum()

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors, weights):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model with weights
    model = sm.WLS(y, X, weights=weights).fit()
    
    return model

# Fit the baseline model for GFTA
gfta_baseline_model = fit_model('GFTA_Standard_scaled', baseline_predictors, df_clean['weights'])

# Fit the expanded model for GFTA (adding canonical_proportion)
gfta_expanded_model = fit_model('GFTA_Standard_scaled', expanded_predictors, df_clean['weights'])

# Log-likelihoods of both models
log_likelihood_baseline = gfta_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = gfta_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(gfta_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(gfta_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of GFTA_Standard")
print(gfta_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of GFTA_Standard")
print(gfta_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


## PPVT-4

### Unweighted, Unscaled (Baseline vs. Expanded)

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['PPVT_Standard', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    return model

# Fit the baseline model for GFTA
ppvt_baseline_model = fit_model('PPVT_Standard', baseline_predictors)

# Fit the expanded model for GFTA (adding canonical_proportion)
ppvt_expanded_model = fit_model('PPVT_Standard', expanded_predictors)

# Log-likelihoods of both models
log_likelihood_baseline = ppvt_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = ppvt_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(ppvt_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(ppvt_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of PPVT_Standard")
print(ppvt_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of PPVT_Standard")
print(ppvt_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


### Weighted, Scaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['PPVT_Standard', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['PPVT_Standard_scaled'] = scaler.fit_transform(df_clean[['PPVT_Standard']])
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Calculate a weight for each child based on their number of canonical and noncanonical clips
# Sum the values from columns '0' and '1' to get the clip count per child
df_clean['clip_count'] = df_clean[['0', '1']].sum(axis=1)

# Normalize the weight column to ensure it sums to 1 (optional step)
df_clean['weights'] = df_clean['clip_count'] / df_clean['clip_count'].sum()

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors, weights):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model with weights
    model = sm.WLS(y, X, weights=weights).fit()
    
    return model

# Fit the baseline model for PPVT
ppvt_baseline_model = fit_model('PPVT_Standard_scaled', baseline_predictors, df_clean['weights'])

# Fit the expanded model for PPVT (adding canonical_proportion)
ppvt_expanded_model = fit_model('PPVT_Standard_scaled', expanded_predictors, df_clean['weights'])

# Log-likelihoods of both models
log_likelihood_baseline = ppvt_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = ppvt_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(ppvt_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(ppvt_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of PPVT_Standard")
print(ppvt_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of PPVT_Standard")
print(ppvt_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


## CTOPP-2

### Unweighted, Unscaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['CTOPP_Elision_Scaled', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    return model

# Fit the baseline model for GFTA
ctopp_baseline_model = fit_model('CTOPP_Elision_Scaled', baseline_predictors)

# Fit the expanded model for GFTA (adding canonical_proportion)
ctopp_expanded_model = fit_model('CTOPP_Elision_Scaled', expanded_predictors)

# Log-likelihoods of both models
log_likelihood_baseline = ctopp_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = ctopp_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(ctopp_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(ctopp_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of CTOPP_Elision_Scaled")
print(ctopp_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of CTOPP_Elision_Scaled")
print(ctopp_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


### Weighted, Scaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['CTOPP_Elision_Scaled', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['CTOPP_Standard_scaled'] = scaler.fit_transform(df_clean[['CTOPP_Elision_Scaled']])
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Calculate a weight for each child based on their number of canonical and noncanonical clips
# Sum the values from columns '0' and '1' to get the clip count per child
df_clean['clip_count'] = df_clean[['0', '1']].sum(axis=1)

# Normalize the weight column to ensure it sums to 1 (optional step)
df_clean['weights'] = df_clean['clip_count'] / df_clean['clip_count'].sum()

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood with weighted regression (WLS)
def fit_model(outcome_var, predictors, weights):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model with weights using WLS (Weighted Least Squares)
    model = sm.WLS(y, X, weights=weights).fit()
    
    return model

# Fit the baseline model for CTOPP
ctopp_baseline_model = fit_model('CTOPP_Standard_scaled', baseline_predictors, df_clean['weights'])

# Fit the expanded model for CTOPP (adding canonical_proportion)
ctopp_expanded_model = fit_model('CTOPP_Standard_scaled', expanded_predictors, df_clean['weights'])

# Log-likelihoods of both models
log_likelihood_baseline = ctopp_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = ctopp_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(ctopp_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(ctopp_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of CTOPP_Elision_Scaled")
print(ctopp_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of CTOPP_Elision_Scaled")
print(ctopp_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


## RWR Accuracy

### Unweighted, Unscaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['avg_rwr_accuracy', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    return model

# Fit the baseline model for GFTA
rwr_baseline_model = fit_model('avg_rwr_accuracy', baseline_predictors)

# Fit the expanded model for GFTA (adding canonical_proportion)
rwr_expanded_model = fit_model('avg_rwr_accuracy', expanded_predictors)

# Log-likelihoods of both models
log_likelihood_baseline = rwr_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = rwr_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(rwr_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(rwr_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of avg_rwr_accuracy")
print(rwr_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of avg_rwr_accuracy")
print(rwr_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


### Weighted, Scaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['avg_rwr_accuracy', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['RWR_Standard_scaled'] = scaler.fit_transform(df_clean[['avg_rwr_accuracy']])
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Calculate a weight for each child based on their number of canonical and noncanonical clips
# Sum the values from columns '0' and '1' to get the clip count per child
df_clean['clip_count'] = df_clean[['0', '1']].sum(axis=1)

# Normalize the weight column to ensure it sums to 1 (optional step)
df_clean['weights'] = df_clean['clip_count'] / df_clean['clip_count'].sum()

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood with weighted regression (WLS)
def fit_model(outcome_var, predictors, weights):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model with weights using WLS (Weighted Least Squares)
    model = sm.WLS(y, X, weights=weights).fit()
    
    return model

# Fit the baseline model for RWR
rwr_baseline_model = fit_model('RWR_Standard_scaled', baseline_predictors, df_clean['weights'])

# Fit the expanded model for RWR (adding canonical_proportion)
rwr_expanded_model = fit_model('RWR_Standard_scaled', expanded_predictors, df_clean['weights'])

# Log-likelihoods of both models
log_likelihood_baseline = rwr_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = rwr_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(rwr_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(rwr_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of avg_rwr_accuracy")
print(rwr_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of avg_rwr_accuracy")
print(rwr_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


## NWR Accuracy

### Unweighted, Unscaled 

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['avg_nwr_accuracy', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood
def fit_model(outcome_var, predictors):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    return model

# Fit the baseline model for GFTA
nwr_baseline_model = fit_model('avg_nwr_accuracy', baseline_predictors)

# Fit the expanded model for GFTA (adding canonical_proportion)
nwr_expanded_model = fit_model('avg_nwr_accuracy', expanded_predictors)

# Log-likelihoods of both models
log_likelihood_baseline = nwr_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = nwr_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(nwr_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(nwr_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of avg_nwr_accuracy")
print(nwr_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of avg_nwr_accuracy")
print(nwr_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")


### Weighted, Scaled

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2

# Load in data
df = pd.read_csv('gfta_ppvt_ctopp_cp_rwr_nwr_me.csv')  # Replace with your actual file path

# Check for missing values and drop rows with missing data in important columns
df_clean = df.dropna(subset=['avg_nwr_accuracy', 'canonical_proportion', 'age_mos', 'gender', 'Maternal_Education_Level'])

# Center and scale 'age_mos' and 'Maternal_Education_Level'
scaler = StandardScaler()
df_clean['NWR_Standard_scaled'] = scaler.fit_transform(df_clean[['avg_nwr_accuracy']])
df_clean['age_mos_scaled'] = scaler.fit_transform(df_clean[['age_mos']])
df_clean['Maternal_Education_Level_scaled'] = scaler.fit_transform(df_clean[['Maternal_Education_Level']])
df_clean['canonical_proportion_scaled'] = scaler.fit_transform(df_clean[['canonical_proportion']])

# Manually dummy code gender as 0 = Male, 1 = Female
df_clean['gender_dummy'] = df_clean['gender'].map({'M': 0, 'F': 1})

# Calculate a weight for each child based on their number of canonical and noncanonical clips
# Sum the values from columns '0' and '1' to get the clip count per child
df_clean['clip_count'] = df_clean[['0', '1']].sum(axis=1)

# Normalize the weight column to ensure it sums to 1 (optional step)
df_clean['weights'] = df_clean['clip_count'] / df_clean['clip_count'].sum()

# Define the predictors for both the baseline and expanded models
baseline_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled']
expanded_predictors = ['age_mos_scaled', 'gender_dummy', 'Maternal_Education_Level_scaled', 'canonical_proportion_scaled']

# Function to fit and return model and log-likelihood with weighted regression (WLS)
def fit_model(outcome_var, predictors, weights):
    X = df_clean[predictors]  # Predictor variables
    X = sm.add_constant(X)  # Add an intercept (constant) to the model
    y = df_clean[outcome_var]  # Outcome variable
    
    # Fit the model with weights using WLS (Weighted Least Squares)
    model = sm.WLS(y, X, weights=weights).fit()
    
    return model

# Fit the baseline model for NWR
nwr_baseline_model = fit_model('NWR_Standard_scaled', baseline_predictors, df_clean['weights'])

# Fit the expanded model for NWR (adding canonical_proportion)
nwr_expanded_model = fit_model('NWR_Standard_scaled', expanded_predictors, df_clean['weights'])

# Log-likelihoods of both models
log_likelihood_baseline = nwr_baseline_model.llf  # Log-likelihood of the baseline model
log_likelihood_expanded = nwr_expanded_model.llf  # Log-likelihood of the expanded model

# Number of parameters in each model
k_baseline = len(nwr_baseline_model.params)  # Number of parameters in baseline model
k_expanded = len(nwr_expanded_model.params)  # Number of parameters in expanded model

# Likelihood ratio test statistic (LRT)
lrt_statistic = 2 * (log_likelihood_expanded - log_likelihood_baseline)

# Degrees of freedom (df) is the difference in the number of parameters
df = k_expanded - k_baseline

# p-value from chi-squared distribution
p_value = chi2.sf(lrt_statistic, df)

# Output the results
print("Baseline Model: Demographic Factors as predictors of avg_nwr_accuracy")
print(nwr_baseline_model.summary())
print("\n")

print("Expanded Model: Demographic Factors and Canonical Proportion as predictors of avg_nwr_accuracy")
print(nwr_expanded_model.summary())

print("\nLikelihood Ratio Test Results:")
print(f"Log-Likelihood of Baseline Model: {log_likelihood_baseline}")
print(f"Log-Likelihood of Expanded Model: {log_likelihood_expanded}")
print(f"Likelihood Ratio Statistic: {lrt_statistic}")
print(f"Degrees of Freedom: {df}")
print(f"P-value: {p_value}")

# Interpretation of p-value
if p_value < 0.05:
    print("The difference between the models is statistically significant.")
else:
    print("The difference between the models is not statistically significant.")
